In [1]:
!pip install nltk scikit-learn pandas

import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from google.colab import files

In [4]:
uploaded = files.upload()

data = pd.read_csv("Students_Attitude_Dataset new (1).csv")
data.head()

Saving Students_Attitude_Dataset new.csv to Students_Attitude_Dataset new (2).csv


,Timestamp,Q1,Q2,Q3,Q4,Q5.1,Q5.2,Q5.3,Q5.4,Q5.5,...,Q8.1.1,Q8.2.1,Q8.3.1,Q8.4.1,Q8.5.1,Q9.1.1,Q9.2.1,Q9.3.1,Q9.4.1,Q9.5.1
0,5.15.2023 8:45:52,1,International Economic Relations,1,3,2,4,2,1,3,...,Agree,Agree,Agree,Disagree,Neutral,Neutral,Agree,Neutral,Disagree,Disagree
1,5.15.2023 9:54:15,1,International Economic Relations,1,2,3,3,3,2,3,...,Agree,Agree,Agree,Neutral,Neutral,Neutral,Neutral,Disagree,Disagree,Neutral
2,5.15.2023 15:17:41,1,International Economic Relations,1,2,4,4,4,2,4,...,Agree,Agree,Agree,Agree,Agree,Agree,Agree,Neutral,Agree,Agree
3,5.15.2023 15:19:02,1,International Economic Relations,1,3,4,4,3,2,4,...,Neutral,Agree,Agree,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral
4,5.15.2023 15:23:54,1,International Economic Relations,2,2,5,4,4,2,4,...,Strongly Agree,Agree,Neutral,Disagree,Strongly Disagree,Neutral,Agree,Neutral,Strongly Disagree,Disagree


In [9]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab') # Add this line to download the missing resource

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    tokens = [w for w in tokens if w.isalpha() and w not in stop_words]
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    return ' '.join(tokens)

# Rename 'Q2' to 'Question' and create a placeholder 'Answer' column
data.rename(columns={'Q2': 'Question'}, inplace=True)
data['Answer'] = "No answer provided for this question." # Placeholder for missing 'Answer' column
data['Processed_Question'] = data['Question'].apply(preprocess)
data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,Timestamp,Q1,Question,Q3,Q4,Q5.1,Q5.2,Q5.3,Q5.4,Q5.5,...,Q8.3.1,Q8.4.1,Q8.5.1,Q9.1.1,Q9.2.1,Q9.3.1,Q9.4.1,Q9.5.1,Answer,Processed_Question
0,5.15.2023 8:45:52,1,International Economic Relations,1,3,2,4,2,1,3,...,Agree,Disagree,Neutral,Neutral,Agree,Neutral,Disagree,Disagree,No answer provided for this question.,international economic relation
1,5.15.2023 9:54:15,1,International Economic Relations,1,2,3,3,3,2,3,...,Agree,Neutral,Neutral,Neutral,Neutral,Disagree,Disagree,Neutral,No answer provided for this question.,international economic relation
2,5.15.2023 15:17:41,1,International Economic Relations,1,2,4,4,4,2,4,...,Agree,Agree,Agree,Agree,Agree,Neutral,Agree,Agree,No answer provided for this question.,international economic relation
3,5.15.2023 15:19:02,1,International Economic Relations,1,3,4,4,3,2,4,...,Agree,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,No answer provided for this question.,international economic relation
4,5.15.2023 15:23:54,1,International Economic Relations,2,2,5,4,4,2,4,...,Neutral,Disagree,Strongly Disagree,Neutral,Agree,Neutral,Strongly Disagree,Disagree,No answer provided for this question.,international economic relation


In [10]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data['Processed_Question'])

In [11]:
def chatbot_response(user_query):
    processed_query = preprocess(user_query)
    query_vec = vectorizer.transform([processed_query])
    similarity = cosine_similarity(query_vec, tfidf_matrix)

    idx = similarity.argmax()
    score = similarity[0][idx]

    if score < 0.3:
        return "I'm sorry, I don't have information about that. Please contact the admin office."

    return data.iloc[idx]['Answer']

In [14]:
print("University FAQ Chatbot (type 'exit' to quit)\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye! ")
        break
    print("Chatbot:", chatbot_response(user_input))

University FAQ Chatbot (type 'exit' to quit)

You: hi
Chatbot: I'm sorry, I don't have information about that. Please contact the admin office.
You: exit
Chatbot: Goodbye! 
